In [ ]:
from ipywidgets import interact, FloatSlider
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Function to generate points based on the provided width, height, dx, and dy
def generate_points(width, height, dx, dy):
    # Calculate the number of points along x and y based on dx and dy
    num_x = max(1, int(np.floor(width / dx)))  # Ensure at least 1 point in x direction
    num_y = max(1, int(np.floor(height / dy)))  # Ensure at least 1 point in y direction

    # Generate the coordinates, centered at (0, 0)
    if num_x == 1:
        x_coords = np.array([0])  # Single column in the middle
    else:
        x_coords = np.linspace(- (num_x // 2) * dx, (num_x // 2) * dx, num_x)

    if num_y == 1:
        y_coords = np.array([0])  # Single row in the middle
    else:
        y_coords = np.linspace(- (num_y // 2) * dy, (num_y // 2) * dy, num_y)

    # Create the 2D grid of points
    points = np.array(np.meshgrid(x_coords, y_coords)).T.reshape(-1, 2)
    return points

# Function to update the plot based on user input
def plot_grid(width, height, dx, dy):
    points = generate_points(width, height, dx, dy)
    
    # Clear the previous plot
    plt.figure(figsize=(6, 6))
    
    # Plot the points
    plt.scatter(points[:, 0], points[:, 1], color='blue')
    
    # Set the limits of the plot based on the board dimensions
    plt.xlim(-width / 2, width / 2)
    plt.ylim(-height / 2, height / 2)
    
    # Set labels and title
    plt.xlabel("Width")
    plt.ylabel("Height")
    plt.title(f"Grid with dx={dx}, dy={dy}, width={width}, height={height}")
    
    # Show the plot
    plt.gca().set_aspect('equal', adjustable='box')
    plt.grid(True)
    plt.show()

# Function to compute the inverse square distance from observation point to each grid point
def compute_sum_at_obs_point(points, obs_point, K):
    # Calculate the distance from the observation point to all grid points
    distances = np.sqrt((points[:, 0] - obs_point[0])**2 + 
                        (points[:, 1] - obs_point[1])**2 + obs_point[2]**2)
    
    # Avoid division by zero for points too close to the observation point
    distances[distances == 0] = np.inf
    
    # Compute the sum of K / distance^2 for all points
    values = K / (distances**2)
    return np.sum(values)

# Function to generate a heatmap by moving the observation point
def generate_heatmap(width, height, dx, dy, K, z_obs, grid_resolution=100):
    # Generate the grid of points on the board
    points = generate_points(width, height, dx, dy)
    
    # Create a grid of observation points in the x-y plane
    x_obs_range = np.linspace(-width / 2, width / 2, grid_resolution)
    y_obs_range = np.linspace(-height / 2, height / 2, grid_resolution)
    
    heatmap = np.zeros((grid_resolution, grid_resolution))
    
    # For each observation point in the x-y grid, compute the sum of values
    for i, x_obs in enumerate(x_obs_range):
        for j, y_obs in enumerate(y_obs_range):
            obs_point = np.array([x_obs, y_obs, z_obs])
            heatmap[j, i] = compute_sum_at_obs_point(points, obs_point, K)
    
    return heatmap, x_obs_range, y_obs_range

# Function to plot the heatmap with adjustable vmax
def plot_heatmap(width, height, dx, dy, K, z_obs, vmax):
    heatmap, x_obs_range, y_obs_range = generate_heatmap(width, height, dx, dy, K, z_obs)
    
    # Plot the heatmap using matplotlib
    plt.figure(figsize=(6, 6))
    plt.imshow(heatmap, extent=[-width / 2, width / 2, -height / 2, height / 2],
               origin='lower', cmap='hot', interpolation='nearest', vmin=0, vmax=vmax)
    
    plt.colorbar(label="illuminance")
    plt.title(f"Heatmap (z={int(z_obs)}, K={int(K)})")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.show()

In [ ]:
# calculate K
phi = 223 # lumen of the LED. Murakami 223 lm, Tsuneoka 4131 lm
unit = 0.01 # 0.001 for mm, 0.01 for cm, 1.0 for m
const = phi / (4*np.pi) / unit**2
const

In [ ]:
# Define sliders for interactive controls using ipywidgets
width_slider = FloatSlider(min=10, max=400, step=5, value=300, description='Width') # v2: 300, Tsuneoka: 260, 
height_slider = FloatSlider(min=10, max=400, step=5, value=200, description='Height') # v2: 200, Tsuneoka: 100, 
dx_slider = FloatSlider(min=1, max=50, step=0.5, value=12.5, description='dx') # v2: 12.5, Tsuneoka: 40, super: 14
dy_slider = FloatSlider(min=1, max=50, step=0.5, value=12.5, description='dy') # v2: 12.5, Tsuneoka: 40, super: 14
K_slider = FloatSlider(min=1, max=3287345, step=1, value=const, description='K') # v2: 177458, Tsuneoka: 3287345, super
z_obs_slider = FloatSlider(min=1, max=100, step=1, value=32, description='z_obs') # v2: 32, Tsuneoka: 5
vmax_slider = FloatSlider(min=1, max=100000, step=10, value=15000, description='vmax')

# Create an interactive plot using interact
interact(plot_heatmap, width=width_slider, height=height_slider, dx=dx_slider, dy=dy_slider, 
         K=K_slider, z_obs=z_obs_slider, vmax=vmax_slider);

In [ ]:
# Define sliders for interactive controls using ipywidgets
width_slider = FloatSlider(min=10, max=500, step=5, value=260, description='Width')
height_slider = FloatSlider(min=10, max=500, step=5, value=100, description='Height')
dx_slider = FloatSlider(min=1, max=50, step=1, value=40, description='dx')
dy_slider = FloatSlider(min=1, max=50, step=1, value=40, description='dy')

# Create an interactive plot using interact
interact(plot_grid, width=width_slider, height=height_slider, dx=dx_slider, dy=dy_slider)